In [ ]:
import numpy as np
import csv
import pandas as pd
import sklearn
from sklearn import model_selection
from sklearn import linear_model
from sklearn import discriminant_analysis

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler

In [ ]:
def get_onsets(file: str) -> list[float]:
    with open(file, 'r') as csvfile:
        read = csv.reader(csvfile)
        return [float(row[0]) for row in read]

hits = get_onsets("../stroop/meas/onsets_t.csv")
miss = get_onsets("../stroop/meas/onsets_f.csv")


In [ ]:
d = pd.read_csv("processed.csv", header=None)
df = d.values[:,1:]

In [ ]:
def gen_data(hits):
    data = []
    for hit in hits:
        start = int((hit-1)*10)
        end = int((hit+15)*10)
        data.append(df[start:end])
    return np.array(data)
hit_data = gen_data(hits)
mis_data = gen_data(miss)
x = np.concatenate((hit_data, mis_data), axis=0)
y = np.array([1]*hit_data.shape[0]+[0]*mis_data.shape[0])

In [ ]:
x_flat = x.reshape(x.shape[0], -1)
x_flat.shape

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(x_flat, y, test_size=0.2, shuffle=True, stratify=y)

In [ ]:
reg = linear_model.LogisticRegression(penalty='l2')
reg.fit(X_train, y_train)
reg.score(X_test, y_test)
reg.score(x_flat,y)

In [ ]:
import pickle
filename = 'glm_model.pkl'
pickle.dump(reg, open(filename, 'wb'))

In [ ]:
clf = sklearn.svm.SVC()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
lda = discriminant_analysis.LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
lda.score(X_test, y_test)

In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.lstm_1 = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=3, batch_first=True)
        self.conv1d = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=5)
        self.fc1 = nn.Linear(3968, 128)
        self.fc2 = nn.Linear(128, output_size)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):  
        x, _ = self.lstm_1(x)
        x = x[:,-1,:]
        x = self.relu(x)
        x = x[:,None,:]
        x = self.relu(self.conv1d(x))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=0.2, shuffle=True, stratify=y)

In [ ]:
label, counts = np.unique(y_train, return_counts=True)
class_weights = [sum(counts)/c for c in counts]
weights = [class_weights[e] for e in y_train]
sampler = WeightedRandomSampler(weights, len(y_train))

In [ ]:
model = LSTMClassifier(x.shape[2], 128, 1)

# Define the loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Create the DataLoader objects for training and testing
train_dataset = TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float())
train_loader = DataLoader(train_dataset, batch_size=64, sampler=sampler)
test_dataset = TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float())
test_loader = DataLoader(test_dataset, batch_size=64)


In [ ]:
# Train the model
n_epochs = 20
for epoch in range(n_epochs):
    for i, (inputs, targets) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs).flatten()
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print("Epoch", epoch+1, "loss:", loss.item())

# Evaluate the model
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, targets in test_loader:
        outputs = model(inputs).flatten()
        predicted = torch.round(outputs)
        print(predicted)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
    accuracy = 100 * correct / total
    print("Test accuracy:", accuracy)